# Chapter 2. The Mathematical Building Blocks of Neural Networks

## Reimplement Neural Network from Scratch Using Tensorflow
- We won't use Keras, just TF API.

In [1]:
import tensorflow as tf
from tensorflow.keras import optimizers
from tensorflow.keras.datasets import mnist
import math
import numpy as np

### Simple Dense Class
- Remember the activation layer: ```output = activation(dot(W, input) + b)```

In [2]:
class NaiveDense:
    def __init__(self, input_size, output_size, activation):
        self.activation = activation
        
        # create a matrix W, of shape (input_size, output_size) initialized with random values
        w_shape = (input_size, output_size)
        w_initial_value = tf.random.uniform(w_shape, minval=0, maxval=1e-1)
        self.W = tf.Variable(w_initial_value)
        
        # create a vector b of shape (output_size,) initialized with 0s
        b_shape = (output_size,)
        b_initial_value = tf.zeros(b_shape)
        self.b = tf.Variable(b_initial_value)
        
    def __call__(self, inputs):
        # apply forward pass
        return self.activation(tf.matmul(inputs, self.W) + self.b)

    @property
    def weights(self):
        # method to retrieve layer's weights
        return [self.W, self.b]

### Simple Sequential Class
- Sequential class to chain the layers in order. It also has a weights property to easily keep track of layers' parameters.

In [3]:
class NaiveSequential:
    def __init__(self, layers):
        self.layers = layers
        
    def __call__(self, inputs):
        x = inputs
        for layer in self.layers:
            x = layer(x)
        return x
    
    @property
    def weights(self):
        weights = []
        for layer in self.layers:
            weights += layer.weights
        return weights

In [4]:
# example 
model = NaiveSequential([
    NaiveDense(input_size=28 * 28, output_size=512, activation=tf.nn.relu),
    NaiveDense(input_size=512, output_size=10, activation=tf.nn.softmax)
])

assert len(model.weights) == 4

2023-01-29 19:47:45.485930: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.


### A Batch Generator
- We need a way to iterate over the MNIST data in min-batches

In [5]:
class BatchGenerator:
    def __init__(self, images, labels, batch_size=128):
        assert len(images) == len(labels)
        self.index = 0
        self.images = images
        self.labels = labels
        self.batch_size = batch_size
        self.num_batches = math.ceil(len(images) / batch_size)
    
    def next(self):
        images = self.images[self.index : self.index + self.batch_size]
        labels = self.labels[self.index : self.index + self.batch_size]
        self.index += self.batch_size
        return images, labels

### Running 1 Training Step
- Steps to updates the weights ofh te model after running it on one batch of data:
    - Compute the predictions of the model for the images in the batch.
    - Computer the loss value for these predictions, given the actual labels.
    - Compute the gradient of the loss with regard to the model's weights.
    - Move the weights by a small amount in the direction opposite to the gradient.
- **Note**: GradientTape will "record” the tensor operations that run inside it, in the form of a computation graph (sometimes called a “tape”). This graph can then be used to retrieve the gradient of any output with respect to any variable or set of variables (instances of the tf.Variable class).

In [6]:
learning_rate = 1e-3

def update_weights(gradients, weights):
    for g, w in zip(gradients, weights):
        w.assign_sub(g * learning_rate)

In [7]:
def one_training_step(model, images_batch, labels_batch):
    # forward pass and record with tape
    with tf.GradientTape() as tape:
        predictions = model(images_batch)
        per_sample_losses = tf.keras.losses.sparse_categorical_crossentropy(labels_batch, predictions)
        average_loss = tf.reduce_mean(per_sample_losses)
    
    # compute gradient of the loss with regard to the weights
    gradients = tape.gradient(average_loss, model.weights)
    
    # update the weights using gradient
    update_weights(gradients, model.weights)
    
    return average_loss

In [8]:
optimizer = optimizers.SGD(learning_rate)

In [9]:
def update_weights(gradients, weights):
    optimizer.apply_gradients(zip(gradients, weights))

### Full Training Loop

In [10]:
def fit(model, images, labels, epochs, batch_size=128):
    for epoch_counter in range(epochs):
        print(f"Epoch {epoch_counter}")
        batch_generator = BatchGenerator(images, labels)
        for batch_counter in range(batch_generator.num_batches):
            images_batch, labels_batch = batch_generator.next()
            loss = one_training_step(model, images_batch, labels_batch)
            if batch_counter % 100 == 0:
                print(f"loss at batch {batch_counter}: {loss:.2f}")

In [11]:
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()
train_images = train_images.reshape((60000, 28 * 28))
train_images = train_images.astype("float32") / 255
test_images = test_images.reshape((10000, 28 * 28))
test_images = test_images.astype("float32") / 255
fit(model, train_images, train_labels, epochs=10, batch_size=128)

11501568/11490434 [==============================] - 1s 0us/step
Epoch 0
loss at batch 0: 8.69
loss at batch 100: 2.19
loss at batch 200: 2.18
loss at batch 300: 2.09
loss at batch 400: 2.19
Epoch 1
loss at batch 0: 1.88
loss at batch 100: 1.84
loss at batch 200: 1.80
loss at batch 300: 1.70
loss at batch 400: 1.79
Epoch 2
loss at batch 0: 1.57
loss at batch 100: 1.55
loss at batch 200: 1.49
loss at batch 300: 1.42
loss at batch 400: 1.48
Epoch 3
loss at batch 0: 1.31
loss at batch 100: 1.31
loss at batch 200: 1.23
loss at batch 300: 1.21
loss at batch 400: 1.26
Epoch 4
loss at batch 0: 1.12
loss at batch 100: 1.14
loss at batch 200: 1.04
loss at batch 300: 1.05
loss at batch 400: 1.10
Epoch 5
loss at batch 0: 0.98
loss at batch 100: 1.00
loss at batch 200: 0.90
loss at batch 300: 0.93
loss at batch 400: 0.98
Epoch 6
loss at batch 0: 0.87
loss at batch 100: 0.90
loss at batch 200: 0.80
loss at batch 300: 0.84
loss at batch 400: 0.90
Epoch 7
loss at batch 0: 0.79
loss at batch 100: 0.82

### Evaluate model

In [12]:
predictions = model(test_images)
predictions = predictions.numpy()
predicted_labels = np.argmax(predictions, axis=1)
matches = predicted_labels == test_labels
print(f"accuracy: {matches.mean():.2f}")

accuracy: 0.82
